### <font color="green">Review Scraping</font>

In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [11]:
link_reviews = pd.read_excel("QualityReviews.xlsx")
print(link_reviews.shape[0])
link_reviews["link"].head(3)

260


0    https://www.booking.com/hotel/vn/florence-dana...
1    https://www.booking.com/hotel/vn/the-backpacke...
2    https://www.booking.com/hotel/vn/siro-house-da...
Name: link, dtype: object

In [48]:
# links = ["https://www.booking.com/hotel/vn/kha-ch-sao-n-paradise.vi.html?label=gen173nr-1FCAEoggI46AdIM1gEaPQBiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuALOh42vBsACAdICJDExZTFmOGQxLTgwMzItNGU2MS1iOTZmLTUwZThiZWM3MGY4ZNgCBeACAQ&aid=304142&ucfs=1&arphpl=1&checkin=2024-05-21&checkout=2024-05-22&dest_id=-3712125&dest_type=city&group_adults=1&req_adults=1&no_rooms=1&group_children=0&req_children=0&hpos=22&hapos=947&sr_order=popularity&srpvid=b81b4a2b6f750050&srepoch=1709463302&all_sr_blocks=56970401_234997815_2_0_0&highlighted_blocks=56970401_234997815_2_0_0&matching_block_id=56970401_234997815_2_0_0&sr_pri_blocks=56970401_234997815_2_0_0__82175000&from=searchresults#tab-reviews", 
#          "https://www.booking.com/hotel/vn/ems-house-ngu-hanh-son.vi.html?label=gen173nr-1FCAEoggI46AdIM1gEaPQBiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuALOh42vBsACAdICJDExZTFmOGQxLTgwMzItNGU2MS1iOTZmLTUwZThiZWM3MGY4ZNgCBeACAQ&aid=304142&ucfs=1&arphpl=1&checkin=2024-05-21&checkout=2024-05-22&dest_id=-3712125&dest_type=city&group_adults=1&req_adults=1&no_rooms=1&group_children=0&req_children=0&hpos=23&hapos=948&sr_order=popularity&srpvid=b81b4a2b6f750050&srepoch=1709463302&all_sr_blocks=1027969207_375543312_4_0_0&highlighted_blocks=1027969207_375543312_4_0_0&matching_block_id=1027969207_375543312_4_0_0&sr_pri_blocks=1027969207_375543312_4_0_0__225000000&from_sustainable_property_sr=1&from=searchresults#tab-reviews",
#          "https://www.booking.com/hotel/vn/zuchis-home-amp-apartment.vi.html?aid=304142&label=gen173nr-1FCAEoggI46AdIM1gEaPQBiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuALOh42vBsACAdICJDExZTFmOGQxLTgwMzItNGU2MS1iOTZmLTUwZThiZWM3MGY4ZNgCBeACAQ&sid=8e0e4f4e21076722b45cf3c399d78a02&all_sr_blocks=867246402_383260669_1_0_0;checkin=2024-05-21;checkout=2024-05-22;dest_id=-3712125;dest_type=city;dist=0;group_adults=1;group_children=0;hapos=145;highlighted_blocks=867246402_383260669_1_0_0;hpos=20;matching_block_id=867246402_383260669_1_0_0;no_rooms=1;req_adults=1;req_children=0;room1=A;sb_price_type=total;sr_order=popularity;sr_pri_blocks=867246402_383260669_1_0_0__38637000;srepoch=1709462169;srpvid=b81b4a2b6f750050;type=total;ucfs=1&#tab-reviews"]

In [12]:
import time
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument('--incognito')
s = Service("C:/Program Files (x86)/chromedriver.exe")
driver = webdriver.Chrome(service=s, options=options)
driver.implicitly_wait(10)

In [50]:
# link_reviews["link"][18::]

In [51]:
hotel_reviews = []
hotel_order = 18

for url in link_reviews["link"][18::]:
    driver.get(url)
    print("Scraping on hotel:", hotel_order)
    sleep(5)

    try:
        # Get HTML tree
        element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[id="bodyconstraint"]')))
        html_of_interest = driver.execute_script('return arguments[0].innerHTML', element)
        soup = BeautifulSoup(html_of_interest, 'lxml')
        sleep(3)
        
        # Get scroll bar of reviews tab
        scr_review_tab = driver.find_element(By.XPATH, '//*[@id="hp-reviews-sliding"]/div[1]')
        sleep(1.5)
        # Check if scroll_review_tab is not None
        if scr_review_tab:
            # Check reviews class
            review_info_row = soup.select('li[class="review_list_new_item_block"]')

            # If review_info_row (element) is not found, skip to the next iteration (hotel)
            if not review_info_row:
                print("Skipping to the next hotel")
                continue

            page_cmt = 1

            while True:
                print("Scraping comments at page:", page_cmt)
                driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", scr_review_tab)
                sleep(2)
                element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[id="bodyconstraint"]')))
                html_of_interest = driver.execute_script('return arguments[0].innerHTML', element)
                soup = BeautifulSoup(html_of_interest, 'lxml')
                review_info_row = soup.select('li[class="review_list_new_item_block"]')

                # Use a try-except block to handle the case when the "Next" button is not found
                for review_item in review_info_row:
                    # reviews_info["reviews"] = review_item.select_one('span[class*="c-review__body"]').text
                    reviews = [span.text for span in review_item.select('span[class*="c-review__body"]')]
                    for review in reviews:
                        reviews_info = {}
                        reviews_info["hotel_name"] = soup.select_one('h2[class="d2fee87262 pp-header__title"]').text
                        reviews_info["review"] = review
                        hotel_reviews.append(reviews_info)

                try:
                    next_page_bt = driver.find_element(By.XPATH, '//*[@id="review_list_page_container"]/div[4]/div/div[1]/div/div[3]/a')
                    # Click on the next button without checking for its enablement
                    next_page_bt.click()
                    page_cmt += 1
                    # Optional: Add a sleep or wait to allow the new content to load
                    sleep(2)
                except NoSuchElementException:
                    # Break the loop if the "Next" button is not found
                    print("Reach the last page!!!")
                    break

            sleep(1.5)

        else:
            # If review_tab is None, skip to the next iteration (hotel)
            print("Skipping to the next hotel")

    except Exception as e:
        print(f"Error processing URL: {url}. Exception: {e}")
    hotel_order += 1

Scraping on hotel: 18
Error processing URL: https://www.booking.com/hotel/vn/gao-ngu-hanh-son.vi.html?label=gen173nr-1FCAEoggI46AdIM1gEaPQBiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuALOh42vBsACAdICJDExZTFmOGQxLTgwMzItNGU2MS1iOTZmLTUwZThiZWM3MGY4ZNgCBeACAQ&aid=304142&ucfs=1&arphpl=1&checkin=2024-05-21&checkout=2024-05-22&dest_id=-3712125&dest_type=city&group_adults=1&req_adults=1&no_rooms=1&group_children=0&req_children=0&hpos=5&hapos=30&sr_order=popularity&srpvid=b81b4a2b6f750050&srepoch=1709462024&all_sr_blocks=919926201_384492822_2_0_0&highlighted_blocks=919926201_384492822_2_0_0&matching_block_id=919926201_384492822_2_0_0&sr_pri_blocks=919926201_384492822_2_0_0__56700000&from_sustainable_property_sr=1&from=searchresults#tab-reviews. Exception: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="hp-reviews-sliding"]/div[1]"}
  (Session info: chrome=122.0.6261.112); For documentation on this error, please visit: https://www.selenium.dev/documentation/w

In [26]:
hotel_reviews = []
hotel_order = 155

for url in link_reviews["link"][155::]:
    driver.get(url)
    print("Scraping on hotel:", hotel_order)
    sleep(5)

    try:
        # Get HTML tree
        element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[id="bodyconstraint"]')))
        html_of_interest = driver.execute_script('return arguments[0].innerHTML', element)
        soup = BeautifulSoup(html_of_interest, 'lxml')
        sleep(3)
        
        # Get scroll bar of reviews tab
        scr_review_tab = driver.find_element(By.XPATH, '//*[@id="hp-reviews-sliding"]/div[1]')
        sleep(1.5)
        # Check if scroll_review_tab is not None
        if scr_review_tab:
            # Check reviews_row class
            review_info_row = soup.select('div[class="d799cd346c"]')
            # If review_info_row (element) is not found, skip to the next iteration (hotel)
            if not review_info_row:
                print("Skipping to the next hotel")
                continue

            page_cmt = 1

            while True:
                print("Scraping comments at page:", page_cmt)
                driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", scr_review_tab)
                sleep(2)

                element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[id="bodyconstraint"]')))
                html_of_interest = driver.execute_script('return arguments[0].innerHTML', element)
                soup = BeautifulSoup(html_of_interest, 'lxml')

                review_info_row = soup.select('div[class="d799cd346c"]')

                # Extract reviews
                for review_item in review_info_row:
                    # Remove hotel's reply
                    remove_owner_reply = review_item.select_one('div[class="c82435a4b8 bed745e358 d01af75260 a18aeea94d d794b7a0f7 f53e278e95"]')
                    if remove_owner_reply:
                        remove_owner_reply.decompose()
                    reviews = [span.text for span in review_item.select('div[class="a53cbfa6de b5726afd0b"] > span')]
                    # Store reviews to a dict and append to hotel_reviews list
                    for review in reviews: 
                        reviews_info = {}
                        reviews_info["hotel_name"] = soup.select_one('h2[class="d2fee87262 pp-header__title"]').text
                        reviews_info["review"] = review
                        hotel_reviews.append(reviews_info)
                # Click on the next page button        
                next_page_bt = driver.find_element(By.XPATH, '//*[@id="reviewCardsSection"]/div[2]/nav/nav/div/div[3]/button')
                if next_page_bt.is_enabled():
                    next_page_bt.click()
                    page_cmt += 1
                    # Add a sleep or wait to allow the new content to load
                    sleep(2)
                else:
                    print("Reach the last page!!!")
                    break   
                        
            sleep(1.5)
        else:
            # If scr_review_tab is None, skip to the next iteration (hotel)
            print("Skipping to the next hotel")

    except Exception as e:
        print(f"Error processing URL: {url}. Exception: {e}")
    hotel_order += 1

Scraping on hotel: 155
Scraping comments at page: 1
Scraping comments at page: 2
Scraping comments at page: 3
Scraping comments at page: 4
Scraping comments at page: 5
Scraping comments at page: 6
Scraping comments at page: 7
Scraping comments at page: 8
Scraping comments at page: 9
Scraping comments at page: 10
Scraping comments at page: 11
Scraping comments at page: 12
Scraping comments at page: 13
Scraping comments at page: 14
Reach the last page!!!
Scraping on hotel: 156
Scraping comments at page: 1
Scraping comments at page: 2
Scraping comments at page: 3
Scraping comments at page: 4
Scraping comments at page: 5
Scraping comments at page: 6
Scraping comments at page: 7
Scraping comments at page: 8
Scraping comments at page: 9
Scraping comments at page: 10
Scraping comments at page: 11
Scraping comments at page: 12
Scraping comments at page: 13
Scraping comments at page: 14
Scraping comments at page: 15
Scraping comments at page: 16
Scraping comments at page: 17
Scraping comments a

In [27]:
# hotelrvnewclass = pd.DataFrame(hotel_reviews)
# hotelrvnewclass.to_excel("NewClassRestHotelReviews.xlsx")